# Genetic Algorithm Feature Selection Demo

This notebook demonstrates the `step_select_genetic_algorithm()` recipe step for intelligent feature selection using genetic algorithms.

## Key Features Demonstrated:
1. **Basic GA feature selection** (ungrouped data)
2. **Advanced GA enhancements** (mandatory/forbidden features, costs, sparsity, warm start)
3. **Adaptive GA parameters** (mutation/crossover adaptation)
4. **Constraint relaxation** (gradual constraint loosening)
5. **Parallel evaluation** (multi-core fitness evaluation)
6. **Ensemble mode** (multiple GA runs with aggregation)
7. **Multi-objective optimization** (NSGA-II for Pareto optimization)
8. **Diversity maintenance** (prevent premature convergence)
9. **Grouped/panel data** (per-group feature selection)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from py_recipes import recipe
from py_recipes.steps import step_select_genetic_algorithm
from py_parsnip import linear_reg, rand_forest
from py_workflows import workflow
from py_yardstick import rmse, mae, r_squared

# Set random seed for reproducibility
np.random.seed(42)

# Style settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Part 1: Ungrouped Data - Basic Feature Selection

We'll create a dataset with:
- 3 important features (strong signal)
- 7 noise features (weak signal)
- Goal: GA should identify the 3 important features

In [ ]:
# Create dataset with known important features
n = 300

# Important features (strong predictors)
x1 = np.random.randn(n)
x2 = np.random.randn(n)
x3 = np.random.randn(n)

# Noise features (weak predictors)
x4 = np.random.randn(n) * 0.1
x5 = np.random.randn(n) * 0.1
x6 = np.random.randn(n) * 0.1
x7 = np.random.randn(n) * 0.1
x8 = np.random.randn(n) * 0.1
x9 = np.random.randn(n) * 0.1
x10 = np.random.randn(n) * 0.1

# Outcome: strong signal from x1, x2, x3
y = 5 * x1 + 3 * x2 + 2 * x3 + np.random.randn(n) * 0.5

data = pd.DataFrame({
    'y': y,
    'x1': x1, 'x2': x2, 'x3': x3,
    'x4': x4, 'x5': x5, 'x6': x6, 'x7': x7,
    'x8': x8, 'x9': x9, 'x10': x10
})

# Split train/test
train = data.iloc[:250]
test = data.iloc[250:]

print(f"Training data: {train.shape}")
print(f"Test data: {test.shape}")
print(f"\nTrue important features: x1, x2, x3")

### Example 1: Basic GA Feature Selection

In [ ]:
# Create recipe with basic GA feature selection
rec_basic = recipe(train)
rec_basic = step_select_genetic_algorithm(
    rec_basic,
    outcome='y',
    model=linear_reg(),
    metric='rmse',
    maximize=False,
    top_n=5,  # Select top 5 features
    population_size=30,
    generations=20,
    cv_folds=3,
    random_state=42,
    verbose=True
)

# Prep and bake
prepped_basic = rec_basic.prep(train)
train_processed = prepped_basic.bake(train)

print(f"\n{'='*60}")
print(f"Selected features: {[col for col in train_processed.columns if col != 'y']}")
print(f"Number of features: {train_processed.shape[1] - 1}")

### Example 2: Mandatory and Forbidden Features

In [ ]:
# Force x1 to be included, forbid x10
rec_constrained = recipe(train)
rec_constrained = step_select_genetic_algorithm(
    rec_constrained,
    outcome='y',
    model=linear_reg(),
    metric='rmse',
    mandatory_features=['x1'],  # Must include x1
    forbidden_features=['x10'],  # Cannot include x10
    top_n=4,
    population_size=20,
    generations=15,
    cv_folds=3,
    random_state=42,
    verbose=True
)

prepped_constrained = rec_constrained.prep(train)
train_constrained = prepped_constrained.bake(train)

print(f"\n{'='*60}")
print(f"Selected features: {[col for col in train_constrained.columns if col != 'y']}")
print(f"x1 included (mandatory): {'x1' in train_constrained.columns}")
print(f"x10 excluded (forbidden): {'x10' not in train_constrained.columns}")

### Example 3: Feature Costs and Sparsity

In [ ]:
# Assign costs to features (higher = more expensive)
costs = {
    'x1': 1.0, 'x2': 1.0, 'x3': 1.0,
    'x4': 5.0, 'x5': 5.0,  # Expensive noise features
    'x6': 2.0, 'x7': 2.0, 'x8': 2.0, 'x9': 2.0, 'x10': 2.0
}

rec_cost = recipe(train)
rec_cost = step_select_genetic_algorithm(
    rec_cost,
    outcome='y',
    model=linear_reg(),
    metric='rmse',
    feature_costs=costs,
    max_total_cost=10.0,  # Budget constraint
    cost_weight=0.3,  # Balance performance vs cost
    sparsity_weight=0.2,  # Prefer fewer features
    top_n=5,
    population_size=25,
    generations=20,
    cv_folds=3,
    random_state=42,
    verbose=True
)

prepped_cost = rec_cost.prep(train)
train_cost = prepped_cost.bake(train)

selected_features = [col for col in train_cost.columns if col != 'y']
total_cost = sum(costs.get(f, 0) for f in selected_features)

print(f"\n{'='*60}")
print(f"Selected features: {selected_features}")
print(f"Total cost: {total_cost:.1f} (max: 10.0)")
print(f"Number of features: {len(selected_features)} (sparsity encouraged)")

### Example 4: Ensemble Mode (Multiple GA Runs)

In [ ]:
# Run 5 GA instances and aggregate results
rec_ensemble = recipe(train)
rec_ensemble = step_select_genetic_algorithm(
    rec_ensemble,
    outcome='y',
    model=linear_reg(),
    metric='rmse',
    top_n=4,
    n_ensemble=5,  # Run 5 GA instances
    ensemble_strategy='voting',  # Aggregate by voting
    ensemble_threshold=0.6,  # Feature must appear in 60%+ of runs
    population_size=20,
    generations=15,
    cv_folds=2,
    random_state=42,
    verbose=True
)

prepped_ensemble = rec_ensemble.prep(train)
train_ensemble = prepped_ensemble.bake(train)

print(f"\n{'='*60}")
print(f"Selected features (ensemble): {[col for col in train_ensemble.columns if col != 'y']}")
print(f"Ensemble size: 5 runs")
print(f"Strategy: voting with 60% threshold")

### Example 5: Diversity Maintenance

In [ ]:
# Maintain population diversity to avoid premature convergence
rec_diversity = recipe(train)
rec_diversity = step_select_genetic_algorithm(
    rec_diversity,
    outcome='y',
    model=linear_reg(),
    metric='rmse',
    top_n=5,
    maintain_diversity=True,
    diversity_threshold=0.3,  # Trigger fitness sharing below this
    fitness_sharing_sigma=0.5,  # Sharing function width
    population_size=30,
    generations=25,
    cv_folds=3,
    random_state=42,
    verbose=True
)

prepped_diversity = rec_diversity.prep(train)
train_diversity = prepped_diversity.bake(train)

print(f"\n{'='*60}")
print(f"Selected features (with diversity): {[col for col in train_diversity.columns if col != 'y']}")
print(f"Diversity maintenance: enabled")

### Example 6: Multi-Objective Optimization (NSGA-II)

In [ ]:
# Optimize multiple objectives simultaneously (performance + sparsity)
rec_nsga2 = recipe(train)
rec_nsga2 = step_select_genetic_algorithm(
    rec_nsga2,
    outcome='y',
    model=linear_reg(),
    metric='rmse',
    use_nsga2=True,  # Enable NSGA-II
    nsga2_objectives=['performance', 'sparsity'],  # Two objectives
    nsga2_selection_method='knee_point',  # Select best trade-off
    population_size=30,  # Must be even for NSGA-II
    generations=20,
    cv_folds=3,
    random_state=42,
    verbose=True
)

prepped_nsga2 = rec_nsga2.prep(train)
train_nsga2 = prepped_nsga2.bake(train)

print(f"\n{'='*60}")
print(f"Selected features (NSGA-II): {[col for col in train_nsga2.columns if col != 'y']}")
print(f"Objectives: performance (RMSE) + sparsity (# features)")
print(f"Selection: knee point (best trade-off)")

### Example 7: Complete Workflow with Evaluation

In [ ]:
# Build complete workflow with GA feature selection
rec_full = recipe(train)
rec_full = step_select_genetic_algorithm(
    rec_full,
    outcome='y',
    model=linear_reg(),
    metric='rmse',
    top_n=5,
    population_size=25,
    generations=20,
    adaptive_mutation=True,  # Adaptive mutation rate
    adaptive_crossover=True,  # Adaptive crossover rate
    cv_folds=3,
    random_state=42,
    verbose=False
)

# Build workflow
wf = workflow().add_recipe(rec_full).add_model(linear_reg())

# Fit and evaluate
fit = wf.fit(train)
fit = fit.evaluate(test)

# Get predictions
train_preds = fit.predict(train)
test_preds = fit.predict(test)

# Calculate metrics
train_rmse = rmse(train['y'].values, train_preds['.pred'].values).iloc[0]['value']
test_rmse = rmse(test['y'].values, test_preds['.pred'].values).iloc[0]['value']
train_r2 = r_squared(train['y'].values, train_preds['.pred'].values).iloc[0]['value']
test_r2 = r_squared(test['y'].values, test_preds['.pred'].values).iloc[0]['value']

print(f"\n{'='*60}")
print("GA Feature Selection + Linear Regression Results:")
print(f"\nTrain RMSE: {train_rmse:.4f}")
print(f"Test RMSE:  {test_rmse:.4f}")
print(f"Train R²:   {train_r2:.4f}")
print(f"Test R²:    {test_r2:.4f}")

# Compare with using all features
wf_all = workflow().add_formula('y ~ .').add_model(linear_reg())
fit_all = wf_all.fit(train)
test_preds_all = fit_all.predict(test)
test_rmse_all = rmse(test['y'].values, test_preds_all['.pred'].values).iloc[0]['value']

print(f"\nComparison (all features): Test RMSE = {test_rmse_all:.4f}")
print(f"Improvement: {((test_rmse_all - test_rmse) / test_rmse_all * 100):.1f}% better")

## Part 2: Grouped/Panel Data - Per-Group Feature Selection

We'll create a panel dataset with multiple groups (e.g., countries) where different features are important for different groups.

In [ ]:
# Create grouped dataset
np.random.seed(42)

groups = ['USA', 'Germany', 'Japan']
n_per_group = 150

data_list = []

for group_idx, group in enumerate(groups):
    # Different features are important for different groups
    x1 = np.random.randn(n_per_group)
    x2 = np.random.randn(n_per_group)
    x3 = np.random.randn(n_per_group)
    x4 = np.random.randn(n_per_group)
    x5 = np.random.randn(n_per_group)
    x6 = np.random.randn(n_per_group)
    
    if group == 'USA':
        # USA: x1, x2, x3 are important
        y = 4 * x1 + 3 * x2 + 2 * x3 + np.random.randn(n_per_group) * 0.5
    elif group == 'Germany':
        # Germany: x2, x4, x5 are important
        y = 3 * x2 + 2 * x4 + 2 * x5 + np.random.randn(n_per_group) * 0.5
    else:  # Japan
        # Japan: x1, x4, x6 are important
        y = 4 * x1 + 2 * x4 + 3 * x6 + np.random.randn(n_per_group) * 0.5
    
    group_data = pd.DataFrame({
        'country': group,
        'y': y,
        'x1': x1, 'x2': x2, 'x3': x3,
        'x4': x4, 'x5': x5, 'x6': x6
    })
    data_list.append(group_data)

panel_data = pd.concat(data_list, ignore_index=True)

# Split train/test
train_panel = panel_data.sample(frac=0.75, random_state=42)
test_panel = panel_data.drop(train_panel.index)

print(f"Panel data shape: {panel_data.shape}")
print(f"Groups: {groups}")
print(f"\nTrue important features by group:")
print("  USA:     x1, x2, x3")
print("  Germany: x2, x4, x5")
print("  Japan:   x1, x4, x6")

### Example 8: Per-Group Feature Selection

We'll use a workflow with nested modeling to select features for each group independently.

In [ ]:
# For grouped data, we'll run GA feature selection per group manually
# (This provides full control and clear results per group)

print(f"\n{'='*60}")
print("Per-Group Feature Selection with GA:")
print(f"{'='*60}\n")

selected_by_group = {}
prepped_by_group = {}

for group in groups:
    # Get group data (without country column)
    group_train = train_panel[train_panel['country'] == group].drop('country', axis=1)
    
    # Run GA for this group
    rec_group = recipe(group_train)
    rec_group = step_select_genetic_algorithm(
        rec_group,
        outcome='y',
        model=linear_reg(),
        metric='rmse',
        top_n=3,
        population_size=20,
        generations=15,
        cv_folds=3,
        random_state=42,
        verbose=False
    )
    
    prepped_group = rec_group.prep(group_train)
    processed_group = prepped_group.bake(group_train)
    selected_features = [col for col in processed_group.columns if col != 'y']
    
    selected_by_group[group] = selected_features
    prepped_by_group[group] = prepped_group
    
    print(f"{group}:")
    print(f"  Selected: {selected_features}")
    print(f"  Expected: ", end="")
    if group == 'USA':
        print("x1, x2, x3")
    elif group == 'Germany':
        print("x2, x4, x5")
    else:
        print("x1, x4, x6")
    print()

# Build per-group models with selected features
print("\nBuilding per-group models with selected features...")
group_models = {}

for group, features in selected_by_group.items():
    formula = 'y ~ ' + ' + '.join(features)
    wf_group = workflow().add_formula(formula).add_model(linear_reg())
    
    # Fit on group training data
    group_train = train_panel[train_panel['country'] == group].drop('country', axis=1)
    group_models[group] = wf_group.fit(group_train)
    
print(f"✓ Trained {len(group_models)} group-specific models")

### Example 9: Evaluate Per-Group Models

In [ ]:
# Evaluate each group's model on test data
print(f"\n{'='*60}")
print("Per-Group Model Performance:")
print(f"{'='*60}\n")

for group in groups:
    # Get group test data
    group_test = test_panel[test_panel['country'] == group].drop('country', axis=1)
    
    # Predict
    group_preds = group_models[group].predict(group_test)
    
    # Calculate metrics
    group_rmse = rmse(group_test['y'].values, group_preds['.pred'].values).iloc[0]['value']
    group_mae = mae(group_test['y'].values, group_preds['.pred'].values).iloc[0]['value']
    group_r2 = r_squared(group_test['y'].values, group_preds['.pred'].values).iloc[0]['value']
    
    print(f"{group}:")
    print(f"  RMSE: {group_rmse:.4f}")
    print(f"  MAE:  {group_mae:.4f}")
    print(f"  R²:   {group_r2:.4f}")
    print()

# Compare with using all features per group
print("\nComparison with using all features:")
for group in groups:
    group_test = test_panel[test_panel['country'] == group].drop('country', axis=1)
    group_train = train_panel[train_panel['country'] == group].drop('country', axis=1)
    
    # Fit model with all features
    wf_all = workflow().add_formula('y ~ .').add_model(linear_reg())
    fit_all = wf_all.fit(group_train)
    preds_all = fit_all.predict(group_test)
    
    rmse_all = rmse(group_test['y'].values, preds_all['.pred'].values).iloc[0]['value']
    rmse_ga = rmse(group_test['y'].values, group_models[group].predict(group_test)['.pred'].values).iloc[0]['value']
    
    improvement = ((rmse_all - rmse_ga) / rmse_all * 100) if rmse_all > rmse_ga else 0
    print(f"{group}: All features RMSE = {rmse_all:.4f}, GA selected RMSE = {rmse_ga:.4f} ({improvement:.1f}% better)")

### Example 10: Visualize GA Evolution History

In [ ]:
# Get GA history from the recipe step (ungrouped example)
rec_vis = recipe(train)
rec_vis = step_select_genetic_algorithm(
    rec_vis,
    outcome='y',
    model=linear_reg(),
    metric='rmse',
    top_n=5,
    population_size=30,
    generations=30,
    cv_folds=3,
    random_state=42,
    verbose=False
)

prepped_vis = rec_vis.prep(train)

# Access the step to get GA history
ga_step = prepped_vis.steps[0]

# Plot evolution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Fitness history
ax1.plot(ga_step._ga_history, linewidth=2, color='steelblue')
ax1.set_xlabel('Generation', fontsize=12)
ax1.set_ylabel('Best Fitness (lower RMSE = better)', fontsize=12)
ax1.set_title('GA Evolution: Fitness Over Time', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Selected features
selected = [col for col in prepped_vis.bake(train).columns if col != 'y']
feature_importance = pd.Series(1, index=selected).sort_values(ascending=True)
feature_importance.plot(kind='barh', ax=ax2, color='coral')
ax2.set_xlabel('Selected', fontsize=12)
ax2.set_title('Final Selected Features', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print(f"\nGA converged: {ga_step._converged}")
print(f"Generations run: {ga_step._n_generations}")
print(f"Final fitness: {ga_step._final_fitness:.4f}")

## Summary

This notebook demonstrated:

### Ungrouped Data Examples:
1. ✅ Basic GA feature selection
2. ✅ Mandatory and forbidden features
3. ✅ Feature costs and sparsity objectives
4. ✅ Ensemble mode (multiple GA runs)
5. ✅ Diversity maintenance
6. ✅ Multi-objective optimization (NSGA-II)
7. ✅ Complete workflow with evaluation

### Grouped Data Examples:
8. ✅ Per-group feature selection
9. ✅ Per-group model evaluation
10. ✅ Visualization of GA evolution

### Key Takeaways:
- GA successfully identifies important features even with noise
- Constraints (mandatory/forbidden) are respected
- Cost and sparsity objectives guide feature selection
- Ensemble mode provides more stable results
- NSGA-II finds optimal trade-offs between objectives
- Per-group selection adapts to group-specific patterns

### Additional Features Not Demonstrated:
- Warm start initialization (importance-based, low-correlation)
- Constraint relaxation (gradual loosening)
- Parallel evaluation (n_jobs > 1)
- Statistical constraints (p-value, VIF, etc.)